## N-Gram Language Modeling

The idea of n-gram language modeling is to calculate the probability of n-gram word w given the history h
$$ P(w|h) = \frac{count(w)}{count(h)}$$ In order to estimate the probability function, we can do maximum likelihood estimation.

Assumption: Markov Chain property, which states that the probability of a word depends only on the previous word (rather than the history or all words precedding the interested word. 
$$ P(w | a) = \frac{P(w,a)}{P(a)} = \frac{count(w, a)}{\sum{count(a)}}$$

Evaluation method: Perplexity

Challenge: 

- sparsity (i.e., count of interested word is significantly smaller than count of history or previous words).
- dependent on training corpus.

Solution to sparsity is smoothing techniques
- simple smoothing: add 1 to the actually count 
- additive smoothing: 
$$ p = \frac{delta + count[prefix][word]} {total[prefix] + delta*vsize} $$
where delta is an additive term, count and total are dictionaries which store count(prefix, word) and count(prefix)
- linear interpolation smoothing

In [1]:
import os
import json
import jsonlines
import numpy as np
from collections import defaultdict

## I. N-Gram Language Modeling

#### Utilities

In [2]:
def load_wikitext(filename='wikitext2-sentencized.json'):
    if not os.path.exists(filename):
        !wget "https://nyu.box.com/shared/static/9kb7l7ci30hb6uahhbssjlq0kctr5ii4.json" -O $filename
    
    datasets = json.load(open(filename, 'r'))
    for name in datasets:
        datasets[name] = [x.split() for x in datasets[name]]
    vocab = list(set([t for ts in datasets['train'] for t in ts]))      
    print("Vocab size: %d" % (len(vocab)))
    return datasets, vocab

def perplexity(model, sequences):
    n_total = 0
    logp_total = 0
    for sequence in sequences:
        logp_total += model.sequence_logp(sequence)
        n_total += len(sequence) + 1  
    ppl = 2 ** (- (1.0 / n_total) * logp_total)  
    return ppl

### Additive Smoothing

In [3]:
class NGramAdditive(object):
    def __init__(self, n, delta, vsize):
        self.n = n
        self.delta = delta
        self.count = defaultdict(lambda: defaultdict(float))
        self.total = defaultdict(float)
        self.vsize = vsize
    
    def estimate(self, sequences):
        for sequence in sequences:
            padded_sequence = ['<bos>']*(self.n-1) + sequence + ['<eos>']
            for i in range(len(padded_sequence) - self.n+1):
                ngram = tuple(padded_sequence[i:i+self.n])
                prefix, word = ngram[:-1], ngram[-1]
                self.count[prefix][word] += 1
                self.total[prefix] += 1
                
    def sequence_logp(self, sequence):
        padded_sequence = ['<bos>']*(self.n-1) + sequence + ['<eos>']
        total_logp = 0
        for i in range(len(padded_sequence) - self.n+1):
            ngram = tuple(padded_sequence[i:i+self.n])
            total_logp += np.log2(self.ngram_prob(ngram))
        return total_logp

    def ngram_prob(self, ngram):
        prefix = ngram[:-1]
        word = ngram[-1]
        prob = ((self.delta + self.count[prefix][word]) / 
                (self.total[prefix] + self.delta*self.vsize))
        return prob

In [5]:
datasets, vocab = load_wikitext()

delta = 0.0005
for n in [2, 3, 4]:
    lm = NGramAdditive(n=n, delta=delta, vsize=len(vocab)+1)  # +1 is for <eos>
    lm.estimate(datasets['train'])

    print("Baseline (Additive smoothing, n=%d, delta=%.4f)) Train Perplexity: %.3f" % (n, delta, perplexity(lm, datasets['train'])))
    print("Baseline (Additive smoothing, n=%d, delta=%.4f)) Valid Perplexity: %.3f" % (n, delta, perplexity(lm, datasets['valid'])))

Vocab size: 33175
Baseline (Additive smoothing, n=2, delta=0.0005)) Train Perplexity: 90.228
Baseline (Additive smoothing, n=2, delta=0.0005)) Valid Perplexity: 525.825
Baseline (Additive smoothing, n=3, delta=0.0005)) Train Perplexity: 26.768
Baseline (Additive smoothing, n=3, delta=0.0005)) Valid Perplexity: 2577.128
Baseline (Additive smoothing, n=4, delta=0.0005)) Train Perplexity: 19.947
Baseline (Additive smoothing, n=4, delta=0.0005)) Valid Perplexity: 9570.901


### Interpolation

In [5]:
class NGramInterpolation(object):
    def __init__(self, n, lambdas, vsize):
        self.n = n
        self.lambdas = lambdas
        self.vsize = vsize
        self.count = defaultdict(lambda: defaultdict(float))
        self.total = defaultdict(float)
    
    def estimate(self, sequences):
        for n in range(1, self.n+1):
            for sequence in sequences:
                padded_sequence = ['<bos>']*(n-1) + sequence + ['<eos>']
                for i in range(len(padded_sequence)-n+1):
                    ngram = tuple(padded_sequence[i:i+n])
                    prefix, word = ngram[:-1], ngram[-1]
                    self.count[prefix][word] += 1
                    self.total[prefix] += 1
    
    def sequence_logp(self, sequence):
        padded_sequence = ['<bos>']*(self.n-1) + sequence + ['<eos>']
        total_logp = 0
        for i in range(len(padded_sequence) - self.n+1):
            ngram = tuple(padded_sequence[i:i+self.n])
            total_logp += np.log2(self.ngram_interp_prob(ngram))
        return total_logp
    
    def ngram_prob(self, ngram):
        prefix = ngram[:-1]
        word = ngram[-1]
        prob = (self.count[prefix][word] / max(self.total[prefix], 1))
        return prob
    
    def ngram_interp_prob(self, sequence):
        ngrams = []
        for i in range(len(sequence)):
            ngrams.append(sequence[i:])
        probs = [self.ngram_prob(ngram) for ngram in ngrams]
        probs.append(1.0/self.vsize)
        interp_prob = sum([lambda_ * prob for lambda_, prob in zip(self.lambdas, probs)])
        return interp_prob

#### Results (showing $\lambda_0,\ldots,\lambda_n$ values):

In [6]:
datasets, vocab = load_wikitext()

Vocab size: 33175


**n = 2**: $\lambda_2$ = 0.5, $\lambda_1$ = 0.4, $\lambda_0$ = 0.1

In [8]:
n = 2
# in the order of lambda2, lambda1, lambda0
lambdas = [0.5, 0.4, 0.1]
lm = NGramInterpolation(n=n, lambdas=lambdas, vsize=len(vocab)+1)  # +1 is for <eos>
lm.estimate(datasets['train'])
print("Interpolation, n=%d, Train Perplexity: %.3f" % (n, perplexity(lm, datasets['train'])))
print("Interpolation, n=%d, Valid Perplexity: %.3f" % (n, perplexity(lm, datasets['valid'])))

Interpolation, n=2, Train Perplexity: 127.724
Interpolation, n=2, Valid Perplexity: 320.489


**n=3**: $\lambda_3$ = 0.1, $\lambda_2$ = 0.5, $\lambda_1$ = 0.3, $\lambda_0$ = 0.1

In [9]:
n = 3
# in the order of lambda3, lambda2, lambda1, lambda0
lambdas = [0.1, 0.5, 0.3, 0.1]
lm = NGramInterpolation(n=n, lambdas=lambdas, vsize=len(vocab)+1)  # +1 is for <eos>
lm.estimate(datasets['train'])
print("Interpolation, n=%d, Train Perplexity: %.3f" % (n, perplexity(lm, datasets['train'])))
print("Interpolation, n=%d, Valid Perplexity: %.3f" % (n, perplexity(lm, datasets['valid'])))

Interpolation, n=3, Train Perplexity: 33.746
Interpolation, n=3, Valid Perplexity: 283.795


**n=4**: $\lambda_4$ = 0.01, $\lambda_3$ = 0.03, $\lambda_2$ = 0.65, $\lambda_1$ = 0.25, $\lambda_0$ = 0.06

In [10]:
n = 4
# in the order of lambda4, lambda3, lambda2, lambda1, lambda0
lambdas = [0.01, 0.03, 0.65, 0.25, 0.06]
lm = NGramInterpolation(n=n, lambdas=lambdas, vsize=len(vocab)+1)  # +1 is for <eos>
lm.estimate(datasets['train'])
print("Interpolation, n=%d, Train Perplexity: %.3f" % (n, perplexity(lm, datasets['train'])))
print("Interpolation, n=%d, Valid Perplexity: %.3f" % (n, perplexity(lm, datasets['valid'])))

Interpolation, n=4, Train Perplexity: 30.419
Interpolation, n=4, Valid Perplexity: 283.463


#### Interpretation:

After carefully choosing $\lambda$ values for each n-gram interpolation model, we found that all three interpolation models outperform the additive smoothing model and performance (measured by validation perplexity) improves as n increases.